In [1]:
import numpy as np
import json

In [2]:
config = {
    "age_starting": 22,
    "age_retirement": 60,
    "age_death": 100,
    "number_of_runs": 10000,
    "amount_starting": 0,
    "contribution_monthly": 3500,
    "retirement_drawdown_monthly": 20000,
    "inflation_mean": 0.06,
    "inflation_std":  0.01,
    "growth_mean": 0.09,
    "growth_std": 0.03,
}

In [3]:
# config = json.load(open('config.json'))['person1']

In [4]:
periods = config['age_death'] - config['age_starting'] + 1
drawdown_start_period = config['age_retirement'] - config['age_starting'] + 1

In [5]:
size = (periods, config['number_of_runs'])

In [6]:
inflation = np.random.normal(config['inflation_mean'], config['inflation_std'], size=size)
growth = np.random.normal(config['growth_mean'], config['growth_std'], size=size)
delta_growth = growth - inflation

In [7]:
scenarios = np.ones(size)

In [8]:
scenarios[0,:] = config['amount_starting']
for p in range(periods)[1:]:
    if drawdown_start_period > p:
        scenarios[p,:] = (scenarios[p-1,:] + config['contribution_monthly']*12) * (1+delta_growth[p,:])
    else:
        scenarios[p,:] = (scenarios[p-1,:] - config['retirement_drawdown_monthly']*12) * (1+delta_growth[p,:])
        
scenarios[scenarios < 0] = 0

In [9]:
percentiles = [10,25,50,75,90]
scenario_stats = np.percentile(scenarios, percentiles, axis=1)
scenario_stats.shape

(5, 79)

In [10]:
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral as palette
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [11]:
# prepare some data
x = np.arange(config['age_starting'], config['age_death']+1)

# create a new plot with a title and axis labels
plot = figure(title="Possible outcomes", x_axis_label="age", y_axis_label="currency", plot_width=950, plot_height=500)

colors = palette[len(percentiles)]

for n, p in enumerate(percentiles): 
    y = scenario_stats[n,:]
    plot.line(x, y, legend_label=f"{p}", line_color=colors[len(percentiles) -1 - n], line_width=2)

# show the results
show(plot)